In [51]:
import sys
import pandas as pd
sys.path.append("C:/Users/ping/MyDrive/py_files/python/py379/")
from myUtils import pickle_load, pickle_dump
pd.set_option('max_colwidth', 12, 'display.max_columns', 18, 'display.width', 1200, 'display.max_rows',100)
INPUT_DIR = 'C:/Users/ping/OneDrive/Documents/jenn_bb_sales'
path_pickle_dump = f'{INPUT_DIR}/'

In [52]:
# read Square csv and pickle df
df1 = pd.read_csv(f'{INPUT_DIR}/items-2021-02-01-2021-10-02.csv')
df2 = pd.read_csv(f'{INPUT_DIR}/items-2021-09-06-2022-09-07.csv')
df = pd.concat([df1, df2])
df = df.drop_duplicates()  # drop overlap between the 2 csv files
df = df.sort_values(by=['Date', 'Time'])
df = df.reset_index(drop=True)  # create new index
pickle_dump(df, path_pickle_dump, 'df_sq_download')
print(f'df.shape: {df.shape}')

df.shape: (18366, 27)


In [53]:
# Los Angeles temperature and precipitation data
# https://www.ncei.noaa.gov/cdo-web/datatools/findstation
dfw = pd.read_csv(f'{INPUT_DIR}/la_weather_2021-01-01_2022-09-28.csv')
dfw = dfw.set_index("DATE")
dfw.index = pd.to_datetime(dfw.index)  # change index to datetime
dfw = dfw.reset_index()  # index in no longer the Date column
dfw['DATE'] = dfw['DATE'].dt.strftime('%Y-%m-%d')  # change Date from datetime to str
dfw.rename(columns={'DATE':'Date'}, inplace=True)  # rename DATE to Date for dataframe merge
dfw

,Date,STATION,NAME,AWND,PGTM,PRCP,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08,DOW
0,2021-01-01,USW00093134,LOS ANGE...,0.22,NaN,0.0,NaN,70,45,250.0,260.0,6.0,11.0,NaN,NaN,1.0,Fri
1,2021-01-02,USW00093134,LOS ANGE...,0.45,NaN,0.0,NaN,66,47,270.0,270.0,6.9,10.1,1.0,NaN,1.0,Sat
2,2021-01-03,USW00093134,LOS ANGE...,1.12,NaN,0.0,NaN,64,50,270.0,260.0,6.9,12.1,1.0,NaN,1.0,Sun
3,2021-01-04,USW00093134,LOS ANGE...,0.89,NaN,0.0,NaN,62,48,260.0,270.0,8.9,14.1,1.0,NaN,1.0,Mon
4,2021-01-05,USW00093134,LOS ANGE...,0.67,NaN,0.0,NaN,63,48,260.0,270.0,8.1,11.0,1.0,1.0,1.0,Tue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,2022-09-24,USW00093134,LOS ANGE...,1.12,NaN,0.0,NaN,86,66,260.0,250.0,8.9,15.0,NaN,NaN,1.0,Sat
632,2022-09-25,USW00093134,LOS ANGE...,1.12,NaN,0.0,NaN,86,66,260.0,260.0,8.9,14.1,NaN,NaN,NaN,Sun
633,2022-09-26,USW00093134,LOS ANGE...,1.12,NaN,0.0,NaN,86,66,270.0,260.0,8.9,14.1,NaN,NaN,NaN,Mon
634,2022-09-27,USW00093134,LOS ANGE...,1.34,1338.0,0.0,NaN,88,66,260.0,260.0,8.1,12.1,1.0,NaN,NaN,Tue


In [54]:
df_merged = df.merge(dfw, how='left', on='Date')
df_merged = df_merged.sort_values(by=['Date', 'Time'])
df_merged.reset_index(drop=True, inplace=True)  # drop current index
pickle_dump(df_merged, path_pickle_dump, 'df_grSale_weather')

In [55]:
df = pickle_load(path_pickle_dump, 'df_grSale_weather')
df

,Date,Time,Time Zone,Category,Item,Qty,Price Point Name,SKU,Modifiers Applied,...,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08,DOW
0,2021-07-08,17:31:07,Pacific ...,Food,choc crx,1.0,Regular,NaN,NaN,...,68,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Thu
1,2021-07-08,17:31:29,Pacific ...,Food,Jarred p...,1.0,Regular,NaN,NaN,...,68,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Thu
2,2021-07-08,17:31:47,Pacific ...,Beverage,sparkling,1.0,Regular,NaN,NaN,...,68,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Thu
3,2021-07-08,17:32:35,Pacific ...,Beverage,sparkling,-1.0,Regular,NaN,NaN,...,68,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Thu
4,2021-07-08,17:32:54,Pacific ...,Food,Jarred p...,-1.0,Regular,NaN,NaN,...,68,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Thu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18361,2022-09-04,11:21:25,Pacific ...,Food,Cinnamon...,1.0,Regular,NaN,NaN,...,77,260.0,260.0,8.1,13.0,NaN,NaN,NaN,Sun
18362,2022-09-04,11:32:24,Pacific ...,Food,Cinnamon...,1.0,Regular,NaN,NaN,...,77,260.0,260.0,8.1,13.0,NaN,NaN,NaN,Sun
18363,2022-09-04,11:39:13,Pacific ...,Food,Danish,2.0,Regular,NaN,NaN,...,77,260.0,260.0,8.1,13.0,NaN,NaN,NaN,Sun
18364,2022-09-06,09:17:15,Pacific ...,None,Croissant,72.0,NaN,NaN,NaN,...,73,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Tue


In [56]:
# dfw.loc[dfw['Date'] == '2021-07-08']
dfw.loc[dfw['Date'] == '2022-09-06']

,Date,STATION,NAME,AWND,PGTM,PRCP,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08,DOW
613,2022-09-06,USW00093134,LOS ANGE...,1.34,NaN,0.0,NaN,93,73,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Tue


In [57]:
df.columns

Index(['Date', 'Time', 'Time Zone', 'Category', 'Item', 'Qty', 'Price Point Name', 'SKU', 'Modifiers Applied', 'Gross Sales', 'Discounts', 'Net Sales', 'Tax', 'Transaction ID', 'Payment ID', 'Device Name', 'Notes', 'Details', 'Event Type', 'Location', 'Dining Option', 'Customer ID', 'Customer Name', 'Customer Reference ID', 'Unit', 'Count', 'Itemization Type', 'STATION', 'NAME', 'AWND', 'PGTM', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT02', 'WT08', 'DOW'], dtype='object')

In [ ]:
for column in df.columns[2::]:
  print(column)

In [107]:
# https://datagy.io/pandas-groupby/
for column in df.columns[2::]:
  s = df[column]
  grouped = s.groupby(s)
  print(f'{column}')
  print('='*20)
  for key in grouped.groups.keys():
    my_count = grouped.get_group(key).count()
    print(f'{key:<40}:{my_count:>7,.0f}')
  print('='*20, '\n')



Time Zone
Pacific Time (US & Canada)              : 18,366

Category
Beverage                                :    197
Coffee & Tea                            :     57
Food                                    : 17,383
Merch                                   :    171
None                                    :    411
RPG                                     :    147

Item
2x choc                                 :    863
Almond box                              :      7
Almond cookies                          :     28
Apple Pie Preorder                      :     22
Apple cider                             :     13
Bento Box                               :     35
Bento Box *10% off! Happy earth month!* :     10
Blueberry Muffin                        :     24
Bluerberry Muffin                       :     82
Bouquet and Candy Box Bundle            :     17
Box                                     :    111
Box *10% off! Happy earth month!*       :      5
Brownie                                 :  

In [64]:
df.iloc[18314]

Date                      2022-09-04
Time                        09:42:35
Time Zone                Pacific ...
Category                         RPG
Item                     Yami Yam...
Qty                              1.0
Price Point Name             Regular
SKU                              NaN
Modifiers Applied                NaN
Gross Sales                   $16.00
Discounts                      $0.00
Net Sales                     $16.00
Tax                            $1.52
Transaction ID           tYnQ3sIs...
Payment ID               Xt5w2uyx...
Device Name                    kiosk
Notes                            NaN
Details                  https://...
Event Type                   Payment
Location                 Bakers B...
Dining Option               For Here
Customer ID              JC58N06Q...
Customer Name            Laly Ruelas
Customer Reference ID            NaN
Unit                              ea
Count                              1
Itemization Type                Item
S

In [ ]:
['Time Zone', ]

In [58]:
# # clean data
# df = pickle_load(path_pickle_dump, 'df_sq_download')
# df_shape_before_clean = df.shape
# # select only these columns
# df = df[['Date', 'Time', 'Category', 'Item', 'Qty',
#        'Price Point Name', 'Gross Sales',
#        'Discounts', 'Net Sales', 'Tax', 
#        'Device Name', 'Notes', 'Event Type', 
#        'Dining Option', 'Customer Name']]
# # strip leading $, convert from object to float
# cols_to_clean = ['Gross Sales', 'Discounts', 'Net Sales', 'Tax']
# for col in cols_to_clean:
#   df[col] = df[col].str.split('$').str[-1]  # strip leading $
#   df[col] = pd.to_numeric(df[col])  # convert from object to float
# df = df.set_index('Date')  # set index to Date column
# df.index = pd.to_datetime(df.index)  # convert Date string to datetime 
# df.Time = pd.to_datetime(df.Time, format= '%H:%M:%S').dt.time  # convert Time string to hour:minute:second
# # remove whole sale customer "Canyon Coffee" "Canyon Coffee"
# print(f'df.shape before removing whole sale customer "Canyon Coffee": {df.shape}')
# df = df[df['Customer Name'] != "Canyon Coffee"]
# print(f'df.shape after removing whole sale customer "Canyon Coffee":  {df.shape}')
# df_shape_after_clean = df.shape
# pickle_dump(df, path_pickle_dump, 'df_clean')
# print(f'df.shape before clean:  {df_shape_before_clean}')
# print(f'df.shape after clean:   {df_shape_after_clean}')